# Create node joint probabilities from the classification tree

JMA 24 March 2025


In [77]:
# Imports from the python standard library
import math, re, os, sys 
from pathlib import Path
import itertools            # to flatten lists
from numpy.random import default_rng

# Import array and dataframe packages
import numpy as np
# import numpy.linalg as la
import pandas as pd
import torch

In [ ]:
# TODO ( Parse classification tree output from 
# Classification_splits.ipynb)

In [ ]:
# dim ig, carb, y ??
joint_cnts = torch.tensor([[[304, 6], [2, 103]], [ [7, 431],[142,5]]])
joint_p = joint_cnts / joint_cnts.sum()
'Joint  p; ', joint_p 

('Joint  p; ',
 tensor([[[0.3040, 0.0060],
          [0.0020, 0.1030]],
 
         [[0.1420, 0.0050],
          [0.0070, 0.4310]]]))

In [80]:
# The implicit prior -- label empirical distribution. 
Py = joint_p.sum(axis=(0,1))
'Raw prior, y: ', Py

('Raw prior, y: ', tensor([0.4550, 0.5450]))

In [81]:
# adjust priors 
adjustment0 = 0.2 / Py[0]
adjustment1 = (1- Py[0]*adjustment0)/Py[1]
# Pc_adjusted = adjustment * Pc[0], (1- Pc[0]*adjustment)
# Pc_adjusted
adjustment = torch.tensor([adjustment0.item(), adjustment1.item()])
'New prior: ',adjustment * Py

('New prior: ', tensor([0.2000, 0.8000]))

In [82]:
# Apply the adjustment to the joint
new_joint_p = joint_p * adjustment.expand(2,2,2)
# Check the new prior
new_joint_p.sum(axis=(0,1))

tensor([0.2000, 0.8000])

In [83]:
# Condition to get p(y | g, c)
# Note, this is just the conditional probabilities at the node leaves. 
y_norm = new_joint_p.sum(2)
Py_given_gc = new_joint_p / y_norm.expand(2,2,2).permute(1,2,0)
Py_given_gc

tensor([[[0.9382, 0.0618],
         [0.0058, 0.9942]],

        [[0.8948, 0.1052],
         [0.0048, 0.9952]]])

In [84]:
# Condition to get (g | c)
# Sum out y: remaining dimensions are g, c
Pgc = new_joint_p.sum(2)
gc_norm = Pgc.sum(1)
Pg_given_c = Pgc / gc_norm.expand(2,2).permute(1,0)
Pg_given_c

tensor([[0.4836, 0.5164],
        [0.0989, 0.9011]])

In [85]:
# Find the preposterior on c
Pc = new_joint_p.sum(axis=(0,2))
Pc

tensor([0.2122, 0.7878])